In [1]:
!pip install cloudscraper

     |████████████████████████████████| 96 kB 1.4 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 


In [2]:
import cloudscraper
from bs4 import BeautifulSoup

url = "https://vinpearl.com/vi/23-dia-diem-du-lich-sai-gon-nghe-la-muon-xach-balo-len-va-di"
scraper = cloudscraper.create_scraper()
rq = scraper.get(url)
doc = BeautifulSoup(rq.text)

doc = doc.find('div', {"class":"container detail news-full"}).find('div', {"class":"content"})

Lưu link ảnh của các địa điểm vào một danh sách

In [3]:
img_tag = doc.find_all('img', {"alt":"dia diem du lich sai gon"})
leftover = doc.find('img', {"alt":"dia diem checkin sai gon"}).get("src")

img_src = []

for i in range(26):
  if (i == 0) or (i == 6) or (i == 8) or (i == 17):
    continue
  elif (i == 5) or (i == 7) or (i == 16):
    pair = [img_tag[i].get("src"), img_tag[i + 1].get("src")]
    img_src.append(pair)
    continue
  elif (i == 23):
    img_src.append(img_tag[i].get("src"))
    img_src.append(leftover)
  else:
    img_src.append(img_tag[i].get("src"))

Lưu miêu tả của từng địa điểm vào một danh sách

In [4]:
p_tag = []
for item in doc.find_all('p')[1:-3]:
  if (">>>" not in item.get_text() and
      "Đặc biệt Vinpearl" not in item.get_text()):
    p_tag.append(item)

index_with_5_des = [4]
index_with_4_des = [11]
index_with_2_des = [2, 6, 7, 8, 10, 18, 19]

description = []
for i in range(23):
  if i in index_with_5_des:
    description.append(p_tag.pop(0).get_text() + p_tag.pop(0).get_text() + p_tag.pop(0).get_text() + p_tag.pop(0).get_text() + p_tag.pop(0).get_text())
  elif i in index_with_4_des:
    description.append(p_tag.pop(0).get_text() + p_tag.pop(0).get_text() + p_tag.pop(0).get_text() + p_tag.pop(0).get_text())
  elif i in index_with_2_des:
    description.append(p_tag.pop(0).get_text() + p_tag.pop(0).get_text())
  else:
    description.append(p_tag.pop(0).get_text() + p_tag.pop(0).get_text() + p_tag.pop(0).get_text())

In [21]:
data = []
h2_tag = doc.find_all('h2')
ul_tag = doc.find_all('ul')[:-1]
ul_tag.pop(5) # We don't want the value of ul_tag[5]
ul_tag.pop(5) # We don't want the value of ul_tag[6]
ul_tag.pop(5) # We don't want the value of ul_tag[7]
ul_tag.pop(5) # We don't want the value of ul_tag[8]

for i in range(23):
    data_itr = dict()

    piece = ul_tag[i].find_all('li')
    data_itr['Place'] = h2_tag[i].get_text().split(". ")[1]
    data_itr['Address'] = piece[0].get_text().split(": ")[1]
    data_itr['Time'] = piece[1].get_text().split(": ")[1]

    try:
      data_itr['Entrance Price'] = piece[2].get_text().split(": ")[1]
    except:
      data_itr['Entrance Price'] = "Miễn phí"
    
    data_itr['Image Source'] = img_src[i]
    data_itr['Description'] = description[i]
    
    data.append(data_itr)

In [19]:
import pandas as pd
df = pd.DataFrame(data)
#df.to_csv("CrawlData_vinpearl_attraction.csv", encoding="utf-8-sig", index = False)
df

,Place,Address,Time,Entrance Price,Image Source,Description
0,Nhà thờ Đức Bà - Biểu tượng của thành phố,"Số 1 Công xã Paris, phường Bến Nghé, thuộc Quậ...",8:30 – 10:30 và 14:30 – 15:30 (Từ thứ 2 đến th...,Miễn phí,https://storage.googleapis.com/vinpearl-data-0...,Nhà thờ Đức Bà là một trong những địa điểm du ...
1,Dinh Độc Lập - Hội trường Thống Nhất,"Số 135 đường Nam Kỳ Khởi Nghĩa, thành phố Hồ C...",7:30 – 11:00 và 13:00 – 16:000 tất cả các ngày...,Từ 3.000 – 30.000 VNĐ/người/lần tùy vào từng đ...,https://storage.googleapis.com/vinpearl-data-0...,Nếu bạn đang tìm kiếm địa điểm du lịch Sài Gòn...
2,Chợ Bến Thành Quận 1 - Khu chợ sầm uất bậc nhấ...,"Giữa các tuyến đường Phan Bội Châu, Lê Thánh T...",7:00 – 19:00,Miễn phí,https://storage.googleapis.com/vinpearl-data-0...,Một trong những địa điểm du lịch Sài Gòn được ...
3,Bưu điện Thành phố Hồ Chí Minh - Địa điểm chec...,"Số 125 Công xã Paris, Bến Nghé, Quận 1",8:00 – 17:00,Miễn phí,https://storage.googleapis.com/vinpearl-data-0...,Bưu điện Thành phố Hồ Chí Minh là một trong nh...
4,Tòa nhà cao nhất Việt Nam - Landmark 81,"Số 720A Điện Biên Phủ, Quận Bình Thạnh, Thành ...",8:30 – 22:00,"810.000 VNĐ/người lớn, 405.000 VNĐ/trẻ em từ 1...",[https://storage.googleapis.com/vinpearl-data-...,Tòa nhà Landmark 81 là một trong những dự án l...
5,Nhà hát Lớn Thành Phố,"Số 7 đường Công Trường Lam Sơn, phường Bến Ngh...",9:00 – 16:30 (ngày thường) và 9:00 – 12:00 (cu...,"Khoảng 700.000 VNĐ/người tùy vào các sự kiện, ...",[https://storage.googleapis.com/vinpearl-data-...,"Ngoài ra, hòa cùng không khí rộn ràng của Tết ..."
6,Bảo tàng lịch sử Việt Nam,"Số 2 đường Nguyễn Bỉnh Khiêm, phường Bến Nghé,...",8:00 – 11:30 và 13:30 – 16:30 (Từ thứ Ba đến C...,"1.000 VNĐ/trẻ em, 2.000 VNĐ/người lớn và 15.00...",https://storage.googleapis.com/vinpearl-data-0...,Gói phòng bao gồm:Mức giá tham khảo: 2.880.000...
7,Bảo tàng Phụ Nữ Nam Bộ - điểm du lịch văn hóa ...,"Số 202 đường Võ Thị Sáu, phường 7, Quận 3, thà...",8:00 – 11:30 và 13:30 – 17:30,30.000 VNĐ/người lớn,https://storage.googleapis.com/vinpearl-data-0...,- Nồng Nàn Sắc XuânVoucher Vinpearl Nồng Nàn S...
8,Bến Nhà Rồng – Bảo tàng Hồ Chí Minh,"Số 1 Nguyễn Tất Thành, Phường 12, Quận 4, Thàn...",7:00 – 11:30 và 13:30 – 17:00 (Từ Thứ Ba đến C...,2.000 VNĐ/lượt,https://storage.googleapis.com/vinpearl-data-0...,Gói phòng bao gồm:Mức giá tham khảo: Từ 3.250....
9,Khu du lịch Suối Tiên - Địa điểm vui chơi hot ...,"120 Xa lộ Hà Nội, phường Tân Phú, Quận 9, thàn...",6:30 – 22:00,"120.000 VNĐ/người lớn, 60.000 VNĐ/trẻ em từ 1 ...",https://storage.googleapis.com/vinpearl-data-0...,- Tết Hoàng KimDu khách đã sẵn sàng đón kỳ ngh...
